## Sentiment analysis on lyrics 

##### Some code was re-used from the notebook here: https://github.com/cristobalvch/Music-Lyrics-NLP/blob/master/lyrics-analysis.ipynb

In [1]:
import pandas as pd
import string 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('vader_lexicon')
nltk.download('punkt')
df_all = pd.read_csv('df_all.csv')
# An example
df_all['song_lyrics_text'][3]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ckeating/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ckeating/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ckeating/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ckeating/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'[Verse 1]Heaven sent you to meIm just hopin’ I dont repeat history[Pre-Chorus]Boy, Im tryna meet your mama on a SundayThen make a lotta love on a Monday (Ah, ah)Never need no (No), no one else, babe’Cause Ill be[Chorus]Switchin them positions for youCookin in the kitchen and Im in the bedroomIm in the Olympics, way Im jumpin through hoopsKnow my love infinite, nothin’ I wouldn’t doThat I wont do, switchin’ for you[Verse 2]Perfect, perfectYoure too good to be true (Youre too good to be true)But I get tired of runnin, fuck itNow, I’m runnin with you (With you)[Pre-Chorus]Said, boy, Im tryna meet your mama on a SundayThen make a lotta love on a Monday (Ah, ah)Never need no (No), no onе else, babeCausе Ill be[Chorus]Switchin them positions for youCookin in the kitchen and Im in the bedroomIm in the Olympics, way Im jumpin through hoopsKnow my love infinite, nothin I wouldnt doThat I wont do, switchin for youCookin in the kitchen and Im in the bedroomIm in the Olympics, way Im jumpin throu

In [2]:
def clean_lyrics(df,column):
    """
    This function cleans the words without importance and fix the format of the  dataframe's column lyrics 
    parameters:
    df = dataframe
    column = name of the column to clean
    """
    df = df
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
    df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
    df[column] = df[column].str.replace("\n"," ").str.replace(r"[^\w\d'\s]+","").str.replace("efil ym fo flah","")
    df[column] = df[column].str.strip()
    return df

In [3]:
df_all_v2 = clean_lyrics(df_all,'song_lyrics_text')
df_all_v2 = df_all_v2.reset_index(drop=True)

<ipython-input-2-223fff7c1943>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
<ipython-input-2-223fff7c1943>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df[column] = df[column].str.replace(r"verse |[1|2|3]|chorus|bridge|outro","").str.replace("[","").str.replace("]","")
<ipython-input-2-223fff7c1943>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
<ipython-input-2-223fff7c1943>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace("\n"," ").s

In [4]:
def lyrics_to_words(document):
    """
    This function splits the text of lyrics to  single words, removing stopwords and doing the lemmatization to each word
    parameters:
    document: text to split to single words
    """
    stop_words = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stopwordremoval = " ".join([i for i in str(document).lower().split() if i not in stop_words])
    punctuationremoval = ''.join(ch for ch in stopwordremoval if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punctuationremoval.split())
    return normalized

In [5]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

In [6]:
words = []

for word in df_all_v2['song_lyrics_text'].tolist():
    words.append(unique(lyrics_to_words(word).split()))
#create the new column with the information of words lists 
df_all_v2['words'] = words

In [7]:
negative = []
neutral = []
positive = []
compound = []

sid = SentimentIntensityAnalyzer()
df_all_v2.dropna(subset=['song_lyrics_text'], inplace=True)
df_all_v2.reset_index(drop=True, inplace=True)

In [8]:
for i in df_all_v2.index:
    scores = sid.polarity_scores(df_all_v2['song_lyrics_text'].iloc[i])
    negative.append(scores['neg'])
    neutral.append(scores['neu'])
    positive.append(scores['pos'])
    compound.append(scores['compound'])

df_all_v2['negative'] = negative
df_all_v2['neutral'] = neutral
df_all_v2['positive'] = positive
df_all_v2['compound'] = compound

In [9]:
df_all_v2.drop(columns=['song_url', 'song_lyrics_text', 'words'], inplace=True)
df_all_v2['stats_hot'] = df_all_v2['stats_hot'].map({False: 0, True: 1})

In [10]:
df_all_v2.to_csv('df_all_cleaned_lyrics.csv')